In [171]:
###
import numpy as np, pandas as pd
import seaborn as sns
import scipy.stats
import matplotlib.pyplot as plt
%matplotlib inline

In [172]:
df = pd.read_pickle("accidents.pkl.gz")
df.head()

,p1,p36,p37,p2a,weekday(p2a),p2b,p6,p7,p8,p9,...,l,n,o,p,q,r,s,t,p5a,region
0,002100160001,4,,2016-01-01,5,0055,1,1,0,2,...,,711403,,Souhlasnýsesměremúseku,Pomalý,554782,451622,GN_V0.1UIR-ADR_410,1,PHA
1,002100160002,4,,2016-01-01,5,0130,1,3,0,2,...,,,,,,,,,1,PHA
2,002100160003,5,,2016-01-01,5,0100,1,2,0,2,...,,,,,,,,,1,PHA
3,002100160004,6,,2016-01-01,5,0120,9,0,0,2,...,,,,,,,,,1,PHA
4,002100160005,6,,2016-01-01,5,2560,2,0,0,2,...,,,,,,,,,1,PHA


Načítame dáta

In [173]:
# vylouceni None hodnot a zaznamov s drogami
df = df[df['p11'].notna()]
df = df[df['p11'] != 4]
df = df[df['p11'] > 0]
df = df[df['p11'] != 5]
df = df[df['p13a'].notna()]
df = df[df['p13b'].notna()]

df_bis = pd.DataFrame({
          'influence': df['p11'] , # pod akym vplyvom bol vodic
          'hard casualties': df['p13a'] + df['p13b'], # pocet usmrtenych alebo tazko zranenych osob
          'deaths': df['p13a'], # smrti
          'casualties': df['p13b'],
         })

df_bis

,influence,hard casualties,deaths,casualties
0,2,0,0,0
1,2,0,0,0
2,2,0,0,0
3,9,0,0,0
5,2,0,0,0
...,...,...,...,...
487153,2,0,0,0
487154,2,0,0,0
487155,2,0,0,0
487157,2,0,0,0


Vytvorenie dataframu so sledovanymi hodnotami, ocistenie hodnot

In [174]:
corr_coefficient, two_tailed_p  = scipy.stats.pearsonr(df_bis["hard casualties"], df_bis["influence"]) # vraci pearson coef + p-value
corr_coefficient

0.019817008094510473

Vykoname korelačnú analýzu pre určenie miery závislosti premenných. Použijem výpočet pearsonovho korelačného koeficienta, ktorý meria štatistický vzťah alebo asociáciu medzi dvoma spojitými premennými.


Hodnota 0.01981 ukazuje na pozitivnu koreláciu medzi skúmanými premennými, tzn. čím vyššie percento alkoholu v krvi vodiča, tým je väčšia šanca výskytu nejakých zranení/podľahnutí (což se dalo očekávat), spôsobených vodičovov zníženov schopnosťou šoférovať vozidlo. Dá se povedať, že $pvalue \le 0.05$, a preto je korelácia významná.

In [175]:
# vytvorit sloupce results, kde je points > median
df_bis['had hard casualties'] = (df_bis['hard casualties'] >
                    0)
# a sloupec manyaces > median
df_bis['very drunk driver'] = (df_bis['influence'] >= 7)
df_bis

,influence,hard casualties,deaths,casualties,had hard casualties,very drunk driver
0,2,0,0,0,False,False
1,2,0,0,0,False,False
2,2,0,0,0,False,False
3,9,0,0,0,False,True
5,2,0,0,0,False,False
...,...,...,...,...,...,...
487153,2,0,0,0,False,False
487154,2,0,0,0,False,False
487155,2,0,0,0,False,False
487157,2,0,0,0,False,False


## $\chi^2$ test
Test statistickej významnosti iným spôsobom. Využijeme **$\chi^2$ chí-kvadrát testu** (chi-square test).
* viník nehody byl pod silným vlivem alkoholu, pokud p11 >= 7,
* těžké zdravotní následky nastaly, pokud došlo k usmrcení či těžkému zranění (p13a + p13b > 0) a


In [176]:
ct = pd.crosstab(df_bis['very drunk driver'], df_bis['had hard casualties'])
ct ###

had hard casualties,False,True
very drunk driver,,
False,264463,9146
True,16492,886


Vytvorím si kontingenčnú tabulku pre všetky štyri kombinácie vstupov.

In [177]:
xi2, p, dof, expected = scipy.stats.chi2_contingency(ct)
p

1.1690773957533103e-34

Funkce spočíta chí-kvadrát štatistiku a p-hodnotu pre test nezávislosti v kontingenčnej tabulke. Táto funkcia vráti viac hodnoty (xi2, p-value, DF, E), nás zaujíma druhá hodnota (p-hodnota).

Hodnota $1.169\cdot10^{-34}$ je oveľa menšia než $0.05$. Preto zamietame nulovú hypotézu a môžeme prehlásit, že korelácia je štatisticky významná medzi množstvom alkoholu v krvi u šoféra vozidla a množstvom ťažkých zranení v nehodách.